**Gaussian Sketching**

In [9]:
import numpy as np

def randomized_compression_gaussian(A, r, rOV, w):
    """
    Compute a compression matrix Q for A using Gaussian Sketching.

    Parameters:
    A (numpy array): Input matrix (m x n)
    r (int): Target rank
    rOV (int): Oversampling parameter
    w (int): Exponent for the power iteration

    Returns:
    Q (numpy array): Compression matrix (m x (r + rOV))
    """
    m, n = A.shape
    d = r + rOV  # Effective reduced dimension

    # Step 1: Draw a Gaussian random matrix Omega_L
    Omega_L = np.random.randn(n, d)

    # Step 2: Compute B = (A A^T)^w A Omega_L
    B = A @ Omega_L  # Initial multiplication: A Omega_L
    for _ in range(w):
        B = (A @ A.T) @ B  # Power iteration: (A A^T) B

    # Step 3: Compute the orthogonal basis Q using QR decomposition
    Q, _ = np.linalg.qr(B)

    return Q

**Subsampled Randomized Hadamard Transform**

In [10]:
import numpy as np
from scipy.linalg import hadamard

def next_power_of_two(n):
    """
    Compute the smallest power of two greater than or equal to n.
    """
    return 2 ** int(np.ceil(np.log2(n)))

def randomized_compression_srht(A, r, rOV, w):
    """
    Compute a compression matrix Q for A using SRHT.

    Parameters:
    A (numpy array): Input matrix (m x n)
    r (int): Target rank
    rOV (int): Oversampling parameter
    w (int): Exponent for the power iteration

    Returns:
    Q (numpy array): Compression matrix (m x (r + rOV))
    """
    m, n = A.shape
    d = r + rOV  # Effective reduced dimension

    # Step 1: Use SRHT to construct Omega_L
    n_padded = next_power_of_two(n)
    if n_padded != n:
        A_padded = np.zeros((m, n_padded))
        A_padded[:, :n] = A  # Fill the padded matrix with the original data
    else:
        A_padded = A

    # Random sign flips
    D = np.diag(np.random.choice([-1, 1], size=n_padded))

    # Hadamard transform
    H = hadamard(n_padded) / np.sqrt(n_padded)  # Normalized Hadamard matrix
    HD = H @ D

    # Subsampling
    S = np.random.choice(n_padded, size=d, replace=False)
    Omega_L = HD[S, :].T  # Transpose to get (n x d)

    # Step 2: Compute B = (A A^T)^w A Omega_L
    B = A @ Omega_L  # Initial multiplication: A Omega_L
    for _ in range(w):
        B = (A @ A.T) @ B  # Power iteration: (A A^T) B

    # Step 3: Compute the orthogonal basis Q using QR decomposition
    Q, _ = np.linalg.qr(B)

    return Q

**Fast Johnson-Lindenstrauss Transform**

In [11]:
import numpy as np
from scipy.fftpack import fft

def randomized_compression_fjlt(A, r, rOV, w):
    """
    Compute a compression matrix Q for A using FJLT.

    Parameters:
    A (numpy array): Input matrix (m x n)
    r (int): Target rank
    rOV (int): Oversampling parameter
    w (int): Exponent for the power iteration

    Returns:
    Q (numpy array): Compression matrix (m x (r + rOV))
    """
    m, n = A.shape
    d = r + rOV  # Effective reduced dimension

    # Step 1: Use FJLT to construct Omega_L
    # Random sign flips
    D = np.diag(np.random.choice([-1, 1], size=n))

    # Fourier transform
    F = fft(np.eye(n)) / np.sqrt(n)  # Normalized Fourier matrix
    FD = F @ D

    # Subsampling
    S = np.random.choice(n, size=d, replace=False)
    Omega_L = FD[S, :].T  # Transpose to get (n x d)

    # Step 2: Compute B = (A A^T)^w A Omega_L
    B = A @ Omega_L  # Initial multiplication: A Omega_L
    for _ in range(w):
        B = (A @ A.T) @ B  # Power iteration: (A A^T) B

    # Step 3: Compute the orthogonal basis Q using QR decomposition
    Q, _ = np.linalg.qr(B)

    return Q

**Clarkson-Woodruff Transform**

In [12]:
import numpy as np

def randomized_compression_cwt(A, r, rOV, w):
    """
    Compute a compression matrix Q for A using CWT.

    Parameters:
    A (numpy array): Input matrix (m x n)
    r (int): Target rank
    rOV (int): Oversampling parameter
    w (int): Exponent for the power iteration

    Returns:
    Q (numpy array): Compression matrix (m x (r + rOV))
    """
    m, n = A.shape
    d = r + rOV  # Effective reduced dimension

    # Step 1: Use CWT to construct Omega_L
    # Random hashing
    h = np.random.randint(0, d, size=n)  # Hash each column to a random bucket
    s = np.random.choice([-1, 1], size=n)  # Random sign flips

    # Construct Omega_L
    Omega_L = np.zeros((n, d))
    for j in range(n):
        Omega_L[j, h[j]] = s[j]

    # Step 2: Compute B = (A A^T)^w A Omega_L
    B = A @ Omega_L  # Initial multiplication: A Omega_L
    for _ in range(w):
        B = (A @ A.T) @ B  # Power iteration: (A A^T) B

    # Step 3: Compute the orthogonal basis Q using QR decomposition
    Q, _ = np.linalg.qr(B)

    return Q

**Examples**

In [17]:
# Example usage
A = np.abs(np.random.rand(100, 50))  # Input matrix (m x n)
r = 5  # Target rank
rOV = 10  # Oversampling parameter
w = 2  # Exponent for power iterations

*Gaussian*

In [18]:
Q_gaussian = randomized_compression_gaussian(A, r, rOV, w)
Q_gaussian

array([[-0.10289712, -0.03085747, -0.05613052, ..., -0.17493657,
        -0.0760745 , -0.29345184],
       [-0.10051681, -0.03963642, -0.04880614, ..., -0.06599275,
        -0.16858263, -0.09311893],
       [-0.09005892,  0.09049653, -0.0116299 , ...,  0.002591  ,
        -0.02453179,  0.06116702],
       ...,
       [-0.09900841,  0.0400544 ,  0.14750179, ...,  0.10848538,
         0.22979927, -0.05624245],
       [-0.09294153,  0.02626261, -0.09097507, ..., -0.02222866,
         0.06070033, -0.08583829],
       [-0.09605035, -0.03315396,  0.062351  , ..., -0.1208717 ,
         0.08309526, -0.09680887]])

*SRHT*

In [ ]:
Q_srht = randomized_compression_srht(A, r, rOV, w)

*FJLT*

In [20]:
Q_fjlt = randomized_compression_fjlt(A, r, rOV, w)
Q_fjlt

array([[-0.10288155+0.00170416j, -0.0148623 -0.10296348j,
        -0.0537929 +0.07163582j, ...,  0.10273087-0.02921677j,
        -0.10532044-0.01485094j, -0.1130027 +0.00957298j],
       [-0.10051072+0.00166255j, -0.04168752+0.02429299j,
        -0.02088152-0.00070977j, ..., -0.0030299 -0.10756103j,
        -0.0835949 -0.00891893j, -0.00088715+0.04213012j],
       [-0.09006187+0.00149038j, -0.0131302 +0.02357831j,
        -0.03283249-0.08988079j, ..., -0.0429979 +0.08213724j,
         0.07124452-0.12199689j,  0.06519093+0.03305138j],
       ...,
       [-0.09898242+0.00164164j,  0.03672836-0.06270043j,
        -0.04461661+0.13969315j, ..., -0.04761941-0.01732438j,
        -0.01709097+0.00362704j, -0.04307726+0.10078172j],
       [-0.09291744+0.00153809j,  0.02276443-0.07163624j,
        -0.07450497+0.0990555j , ...,  0.0442376 +0.02845435j,
        -0.00695262+0.08475662j, -0.00861068-0.13572811j],
       [-0.09604534+0.00159796j, -0.01063441-0.00486696j,
        -0.03178966-0.03599645

*CWT*

In [22]:
Q_cwt = randomized_compression_cwt(A, r, rOV, w)
Q_cwt

array([[-0.10289761, -0.12247631, -0.02330746, ..., -0.06047213,
        -0.10351672, -0.02753897],
       [-0.10052509, -0.04797562,  0.06794174, ..., -0.13295271,
        -0.12006892, -0.01799292],
       [-0.09006596,  0.02894567, -0.17151377, ..., -0.19532633,
         0.10345272, -0.01556757],
       ...,
       [-0.09900159,  0.07242994, -0.13765907, ..., -0.01618775,
         0.02742813, -0.02568031],
       [-0.0929368 , -0.06095951,  0.02228634, ...,  0.11496098,
         0.01864464,  0.10346116],
       [-0.09605088, -0.02793842,  0.00676345, ...,  0.10906788,
        -0.03879134, -0.09277804]])